In [13]:
from quartic_sdk import APIClient
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

In [14]:
# intialize a Quartic Platform API client  
client = APIClient(host='https://qa.quartic.ai', username='Prathik', password='Prathik1')

In [15]:
assets = client.assets()

In [16]:
asset = assets.get("name", "Asset Cont replay 01")
asset_tags = client.tags(asset_id=asset.id)

In [17]:
from datetime import datetime

stop_time = int(datetime.now().timestamp() * 1000) # get current time in millisec unix timestamp
start_time = stop_time - 30 * 60 * 1000 # timestamp 30 mins ago
asset_data_itr = asset.data(start_time=start_time, stop_time=stop_time)

df = pd.DataFrame()
for next_df in asset_data_itr:
    df = pd.concat([df, next_df])
df

,39552,39553,39554,39555,39556,39557,39558,39559,39560,39561,...,40046,39543,39544,39545,39546,39547,39548,39549,39550,39551
1616615594000,140.55900,4,4,9.9650,9.8433,889,35.520,41.066,38.375,30.566,...,4.216926,4,2114,4,46.442,2.136,2621.9001,7.824,10.162,166.44800
1616615595000,140.54001,5,5,9.9600,9.8492,890,35.520,41.066,38.375,30.566,...,5.175235,5,2115,5,46.442,2.136,2621.8801,7.818,10.214,166.44100
1616615596000,140.52300,6,6,9.9628,9.8453,891,35.520,41.066,38.375,30.566,...,6.133544,6,2116,6,46.442,2.136,2621.6000,7.858,10.196,166.37200
1616615597000,140.48200,7,7,9.9696,9.8479,892,35.520,41.066,38.375,30.566,...,7.091853,7,2117,7,46.442,2.136,2620.1199,7.777,10.150,166.28000
1616615598000,140.49800,8,8,9.9690,9.8544,893,35.520,41.066,38.375,30.566,...,8.050162,8,2118,8,46.442,2.136,2620.5200,7.745,10.174,166.31300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616617376000,130.48500,14,14,9.9617,9.8501,2622,35.437,40.879,38.520,30.555,...,NaN,14,247,14,47.075,2.180,2483.8400,7.838,10.167,156.38501
1616617377000,130.59800,1,1,9.9592,9.8532,2623,35.437,40.879,38.520,30.555,...,NaN,1,248,1,47.075,2.180,2483.8000,7.709,10.182,156.44600
1616617378000,130.45801,2,2,9.9722,9.8564,2624,35.437,40.879,38.520,30.555,...,NaN,2,249,2,47.075,2.180,2482.2800,8.020,10.171,156.44100
1616617379000,130.50800,3,3,9.9717,9.8481,2625,35.437,40.879,38.520,30.555,...,NaN,3,250,3,47.075,2.180,2482.0000,7.826,10.160,156.39400


In [18]:
df = df.dropna(axis=1)

In [19]:
target_col = "39543"
feature_cols = ["39553", "39554", "39555", "39556", "39557", "39558"]

In [20]:
df_X = df[feature_cols]
X = df_X.to_numpy()
df_y = df[[target_col]]
y = df_y.to_numpy()

In [21]:
# X_pca = PCA(n_components=2, random_state=123).fit_transform(X)

pca_model = PCA(n_components=2, random_state=123).fit(X)
X_pca = pca_model.transform(X)

In [22]:
reg = LinearRegression().fit(X_pca, y)
preds = reg.predict(X_pca)
preds.flatten()

array([4.17385773, 5.1241462 , 6.07443476, ..., 2.20401517, 3.15430384,
       1.25360719])

In [23]:
from quartic_sdk.model import BaseQuarticModel

class LinearRegressionModel(BaseQuarticModel):
    def __init__(self, name, description, log_level, model, pca_model):
        self.model = model
        self.pca_model = pca_model
        super().__init__(name=name, description=description, log_level=log_level)
        
    def predict(self, input_df: pd.DataFrame):
        self.log.info("Predicting...")
        X = input_df.to_numpy()
        X_pca = self.pca_model.transform(X)
        y_preds = self.model.predict(X_pca)
        return pd.Series(y_preds.flatten())

In [24]:
lr_pca_model = LinearRegressionModel(name="lr_pca_3",
                                     description="model trained with data transformed by PCA",
                                     log_level="INFO",
                                     model=reg,
                                     pca_model=pca_model)

lr_pca_model.save(client=client, output_tag_name='lr_pca_3', feature_tags=feature_cols, target_tag=target_col, test_df=df_X)

Exception: Failed to Save model: {"code":400,"message":"A model with name \"lr_pca_3\" already exists, try different model name"}